# Development Notebook for extracting icebergs from DEMs

by Jessica Scheick

Workflow based on previous methods and code developed by JScheick for Scheick et al 2019 *Remote Sensing*.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
import hvplot.xarray
# import hvplot.pandas
import holoviews as hv
hv.extension('bokeh','matplotlib')
from holoviews import dim, opts
import datetime as dt
import os
import panel as pn
pn.extension()
import pyproj

In [ ]:
%load_ext autoreload
# from icebath.core import berggdf as bdf
import icebath as icebath
from icebath.core import build_xrds
from icebath.utils import raster_ops as raster_ops
from icebath.utils import vector_ops as vector_ops
from icebath.core import fl_ice_calcs as icalcs
from icebath.core import build_gdf
%autoreload 2

## Read in DEMs and apply corrections (tidal, geoid)

In [ ]:
#Ilulissat Isfjord Mouth, resampled to 50m using CHANGES
# ds = build_xrds.xrds_from_dir('/home/jovyan/icebath/notebooks/supporting_docs/Elevation/ArcticDEM/Regridded_50m_tiles/n69w052/')

# Ilulissat Isfjord Mouth, original 2m (the files from CHANGES seem much smaller than those from Kane/Pennell. 
# data = xr.open_rasterio('/home/jovyan/icebath/notebooks/supporting_docs/Elevation/ArcticDEM/2m_tiles/n69w052/SETSM_W1W1_20100813_102001000E959700_102001000ECB6B00_seg1_2m_v3.0_dem.tif')
ds = build_xrds.xrds_from_dir('/Users/jessica/projects/bathymetry_from_bergs/DEMs/2m/')
# ds = build_xrds.xrds_from_dir('/home/jovyan/icebath/notebooks/supporting_docs/Elevation/ArcticDEM/2m_tiles/')

In [ ]:
scrolldem = ds['elevation'].hvplot.image(x='x', y='y',datashade=False, rasterize=True, aspect='equal', cmap='magma', dynamic=True,
                       xlabel="x (km)", ylabel="y (km)", colorbar=True) #turn off datashade to see hover values + colorbar
scrolldem

In [ ]:
ds

In [ ]:
ds.elevation.min(['x','y'], skipna=True)

In [ ]:
ds.elevation.min(['x','y'], skipna=True)

In [ ]:
ds.elevation.max(['x','y'], skipna=True)

In [ ]:
ds.elevation.max(['x','y'], skipna=True)

### Get and Apply Land Mask
**Note: requires a shapefile of the land areas in the ROI**

The default is to use a shapefile of Greenland: `shpfile='/home/jovyan/icebath/notebooks/supporting_docs/Land_region.shp'`, but an alternative file can be specified.

Underlying code is based on: https://gis.stackexchange.com/questions/357490/mask-xarray-dataset-using-a-shapefile
Other results used rioxarray (which isn't on my current working environment), and my previous work did it all manually with gdal.

In [ ]:
# shpfl.geometry.plot()

In [ ]:
# ds.bergxr.get_mask(req_dim=['x','y'], req_vars=None, name='land_mask')
ds.bergxr.get_mask(req_dim=['x','y'], req_vars=None, name='land_mask', shpfile='/Users/jessica/mapping/shpfiles/Greenland/Land_region/Land_region.shp')
# ds.land_mask.plot()
ds['elevation'] = ds['elevation'].where(ds.land_mask == True)

### Apply Geoid Correction
ArcticDEMs come as ellipsoidal height. They are corrected to geoidal height according to geoid_ht = ellipsoid - geoid_offset where geoid_offset is taken from BedMachine v3 and resampled in Xarray (using default "linear" interpolation for multidimensional arrays) to the resolution and extent of the region's dataset.

In [ ]:
ds = ds.bergxr.to_geoid()

### Apply Tidal Correction

Uses Tyler Sutterly's pyTMD library

In [ ]:
ds=ds.bergxr.tidal_corr(loc=["JI"], model_path='/Users/jessica/computing/tidal_model_files')

In [ ]:
# # test to make sure that if you already have a tidal correction it won't reapply it, and test that it will return the tides if you don't have an elevation entered
# ds=ds.bergxr.tidal_corr(loc=["JI"])
# ds=ds.bergxr.tidal_corr(loc=["JI"]) # results in assertion error

# ds.attrs['offset_names'] = ('random')
# ds=ds.bergxr.tidal_corr(loc=["JI"]) # results in longer attribute list

In [ ]:
# # go directly to icalcs function, called under the hood above, if you want to see plots
# tides = icalcs.predict_tides(loc='JI',img_time=ds.dtime.values[0], model_path='/home/jovyan/pyTMD/models',
#                     model='AOTIM-5-2018', epsg=3413, plot=True)
# tides[2]

## Extract Icebergs from DEM and put into Geodataframe

In [ ]:
# this would be a good step to try parallelizing with Dask on - the operation can be done on multiple images at once!
gdf = build_gdf.xarray_to_gdf(ds)

In [ ]:
gdf.groupby('date').berg_poly.plot()

In [ ]:
# This requires geoviews be installed, and their install pages have warning if your environment uses conda-forge
# libraries and it won't resolve the environment with a conda install, so I'll need to create a new test env to try this
# bergs = gdf.hvplot()
# bergs

In [ ]:
# scrolldems*bergs

## Visualizing Iceberg Outlines for a Single DEM
Some attempts at doing this with Holoviews, including to try and have it with a slider bar, are in the misc_dev_notes_notebook, as is code for plotting a histogram of iceberg areas (originally meant to be part of the below plot and not updated to be independent)

In [ ]:
timei=1
print(ds['dtime'].isel({'dtime':timei}))
dem = ds.isel({'dtime':timei})

In [ ]:
im = dem.elevation.values

In [ ]:
# Plot objectives: show DEM, land mask, iceberg outlines. 2nd plot with just orig DEM?
fig = plt.figure(figsize=(12,12)) # width, height in inches
# gs = gridspec.GridSpec(ncols=1, nrows=2, figure=fig)
gs=fig.add_gridspec(3,1, hspace=0.3) # nrows, ncols

# DEM plot
axDEM = plt.subplot(gs[0:2,0])
dem.elevation.plot.pcolormesh(ax=axDEM, 
                              vmin=-10, vmax=75, cmap='magma', # vmin and vmax set the colorbar limits here
                              xscale='linear', yscale='linear',
                              cbar_kwargs={'label':"Elevation (m amsl)"})

# land mask
landcm = mpl.colors.ListedColormap([(0.5, 0.35, 0.35, 1.), (0.5, 0., 0.6, 0)])
dem.land_mask.plot(ax=axDEM, cmap=landcm, add_colorbar=False)

# iceberg contours - ultimately add this from geodataframe
# dem.elevation.plot.contour(ax=axDEM, levels=[threshold], colors=['gray'])
# Note: dem.elevation.plot.contour(levels=[threshold], colors=['gray']) will show the plot, but you can't
# add it to these axes and then show it inline from a second cell
# I'm not entirely sure this is plotting what I think; it's also not actually plotting the contoured data 
gdf.loc[gdf['date']==ds.dtime.isel({'dtime':timei}).values].berg_poly.plot(ax=axDEM,
                                                                          linestyle='-',
                                                                          linewidth=2,
                                                                          edgecolor='gray',
                                                                          facecolor=(0,0,0,0))


xmin = -250000
xmax = -232750
ymin = -2268250
ymax = -2251000
# xmin = -235000 #zoom in to figure out empty iceberg DEM during gdf generation
# xmax = -233000
# ymin = -2257500
# ymax = -2255000
while (xmin-xmax) != (ymin-ymax):
    print("modify your x and y min/max to make the areas equal")
    break
    
axDEM.set_aspect('equal')
axDEM.set_xlim(xmin, xmax)
axDEM.set_ylim(ymin, ymax)

axDEM.set_xlabel("x (km)")
axDEM.set_ylabel("y (km)")


plt.show()

In [ ]:
# Note: gdf['date']==timei is returning all false, so the datetimes will need to be dealt with to get the areas from the geometry column
# areas = gdf.loc[:, gdf['date']==timei].geometry.area()

In [ ]:
# Plot objectives: show DEM, land mask, iceberg outlines. 2nd plot with just orig DEM? 3rd plot with histogram...
fig = plt.figure(figsize=(12,12)) # width, height in inches
# gs = gridspec.GridSpec(ncols=1, nrows=2, figure=fig)
gs=fig.add_gridspec(3,1, hspace=0.3) # nrows, ncols

# DEM plot
axDEM = plt.subplot(gs[0:2,0])
dem.elevation.plot.pcolormesh(ax=axDEM, 
                              vmin=-10, vmax=75, cmap='magma', # vmin and vmax set the colorbar limits here
                              xscale='linear', yscale='linear',
                              cbar_kwargs={'label':"Elevation (m amsl)"})

# land mask
landcm = mpl.colors.ListedColormap([(0.5, 0.35, 0.35, 1.), (0.5, 0., 0.6, 0)])
dem.land_mask.plot(ax=axDEM, cmap=landcm, add_colorbar=False)

# iceberg contours - ultimately add this from geodataframe
# dem.elevation.plot.contour(ax=axDEM, levels=[threshold], colors=['gray'])
# Note: dem.elevation.plot.contour(levels=[threshold], colors=['gray']) will show the plot, but you can't
# add it to these axes and then show it inline from a second cell
# I'm not entirely sure this is plotting what I think; it's also not actually plotting the contoured data 
gdf.loc[gdf['date']==ds.dtime.isel({'dtime':timei}).values].berg_poly.plot(ax=axDEM,
                                                                          linestyle='-',
                                                                          linewidth=2,
                                                                          edgecolor='gray',
                                                                          facecolor=(0,0,0,0))


# xmin = -250000
# xmax = -232750
# ymin = -2268250
# ymax = -2251000
xmin = -235000 #zoom in to figure out empty iceberg DEM during gdf generation
xmax = -233000
ymin = -2257500
ymax = -2255000
while (xmin-xmax) != (ymin-ymax):
    print("modify your x and y min/max to make the areas equal")
    break
    
axDEM.set_aspect('equal')
axDEM.set_xlim(xmin, xmax)
axDEM.set_ylim(ymin, ymax)

axDEM.set_xlabel("x (km)")
axDEM.set_ylabel("y (km)")


plt.show()

In [ ]:
# also, look into a sea level adjustment for all bergs (many are nan)
gdf.sl_adjust

## Compute Water Depths on Icebergs

In [ ]:
gdf.berggdf.calc_filt_draft()

In [ ]:
gdf.berggdf.calc_rowwise_medmaxmad('filtered_draft')

In [ ]:
gdf.berggdf.wat_depth_uncert('filtered_draft')

In [ ]:
print(np.median(gdf.filtered_draft_med.values))
print(np.nanmax(gdf.filtered_draft_med.values))

In [ ]:
shpgdf = gdf.copy(deep=True)
shpgdf['year'] = shpgdf['date'].dt.year.astype(int)

del shpgdf['date']
del shpgdf['DEMarray']
del shpgdf['filtered_draft']

# NOTE: need to rename columns due to name length limits for shapefile; otherwise,
# all ended up as "filtered_#"

shpgdf.to_file("/Users/jessica/projects/bathymetry_from_bergs/prelim_results/pot_icebergs_JI_seg.shp")

In [ ]:
mmm(gdf.berg_poly.area)

In [ ]:
def mmm(vals): # mmm = min, med, max
    print(np.nanmin(vals))
    print(np.nanmedian(vals))
    print(np.nanmax(vals))